In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import plotly.express as px
import time

plt.style.use('fivethirtyeight')
color_pal = plt.rcParams["axes.prop_cycle"].by_key()["color"]

import yfinance as yf
from fredapi import Fred

import sys
sys.path.append("/Users/jtchoi/Library/CloudStorage/GoogleDrive-jungtaek0227@gmail.com/My Drive/quant/Quant-Project/quant")
from backtest.metric import *
from econ_regime.econ_regime import *

In [2]:
sp500 = yf.download('SPY')['Adj Close']
sp500.index = pd.to_datetime(pd.to_datetime(sp500.index).date)
sp500 = sp500.resample('M').last()
sp500 = sp500.iloc[:-1]

[*********************100%***********************]  1 of 1 completed


In [3]:
sp500 = sp500.loc['2011':'2022'].resample('M').last()

In [4]:
regime_df = business_cycle().shift(1)
regime_df = regime_df.loc['2011':'2022']
regime_df

,deflation,inflation,recovery,expansion
2011-01-01,0.0,0.0,1.0,0.0
2011-02-01,0.0,0.0,0.0,1.0
2011-03-01,0.0,0.0,0.0,1.0
2011-04-01,0.0,0.0,0.0,1.0
2011-05-01,0.0,0.0,0.0,0.0
...,...,...,...,...
2022-08-01,0.0,1.0,0.0,0.0
2022-09-01,0.0,1.0,0.0,0.0
2022-10-01,0.0,1.0,0.0,0.0
2022-11-01,0.0,1.0,0.0,0.0


In [5]:
regime_df.index = sp500.index
regime_df = pd.concat([sp500.pct_change(), regime_df], axis=1).dropna()
regime_df.columns = ['spy', 'deflation', 'inflation', 'recovery', 'expansion']
regime_df

,spy,deflation,inflation,recovery,expansion
2011-02-28,0.034737,0.0,0.0,0.0,1.0
2011-03-31,0.000120,0.0,0.0,0.0,1.0
2011-04-30,0.028962,0.0,0.0,0.0,1.0
2011-05-31,-0.011215,0.0,0.0,0.0,0.0
2011-06-30,-0.016870,0.0,0.0,0.0,0.0
...,...,...,...,...,...
2022-08-31,-0.040802,0.0,1.0,0.0,0.0
2022-09-30,-0.092446,0.0,1.0,0.0,0.0
2022-10-31,0.081276,0.0,1.0,0.0,0.0
2022-11-30,0.055592,0.0,1.0,0.0,0.0


In [6]:
_list = ['returns', 'CAGR', 'volatility', 'MDD','sharp']
sp_bt = Metric(portfolio=(1 + sp500.pct_change()).cumprod(), freq='month').numeric_metric()

_dict = {}
for i in _list:
    _dict[i] = [sp_bt[i]]

sp_bt = pd.DataFrame(_dict, index=['spy']).T
sp_bt

,spy
returns,360.91
CAGR,11.29
volatility,0.14
MDD,-0.24
sharp,0.50


In [7]:
_list = ['returns', 'CAGR', 'volatility', 'MDD', 'sharp']
_regime = ['deflation',	'inflation','recovery',	'expansion']

_dict = []
for i in _regime:
    temp = {}
    for j in _list:
        temp[j] = [Metric(portfolio=(1 + regime_df[regime_df[i]==1].spy).cumprod(), freq='month').numeric_metric()[j]]
    _dict.append(pd.DataFrame(temp, index=[f'{i}']).T)
    
temp = pd.concat(_dict, axis=1)
final = pd.concat([sp_bt, temp], axis=1)
final

,spy,deflation,inflation,recovery,expansion
returns,360.91,137.77,93.63,179.43,182.10
CAGR,11.29,12.36,-12.33,21.52,22.81
volatility,0.14,0.18,0.23,0.11,0.08
MDD,-0.24,-0.19,-0.13,-0.07,-0.06
sharp,0.50,0.46,-0.70,1.64,2.31


In [8]:
!pwd
!ls

/Users/jtchoi/Library/CloudStorage/GoogleDrive-jungtaek0227@gmail.com/My Drive/quant/Quant-Project/quant
__pycache__                     make_price_universe.ipynb
alter_with_equity.csv           presentation.ipynb
asset_universe.csv              price
backtest                        price_info_df.csv
bs_with_beta.ipynb              price_universe.csv
bs_with_momentum.ipynb          result
bs_with_sp500.ipynb             sp500_clustering.json
bs_with_vol.ipynb               story_line_body_1.ipynb
business_cycle.csv              strategy
business_cycle.ipynb            test_beta.ipynb
econ_regime                     test_cs.ipynb
econ_regime.csv                 test_momentum.ipynb
equity_universe.csv             test_ts.ipynb
factor_selection(monthly).ipynb test_value.ipynb
factor_selection(regime).ipynb  test_volatility.ipynb
financial_report.ipynb          total_regime.csv
fredapikey.txt                  vaa_momentum.ipynb
load_price_data.ipynb


In [9]:
sp500_cluster = pd.read_json('./sp500_clustering.json')
sp500_cluster
sp500_cluster.index = sp500_cluster.date_time
sp500_cluster.drop(columns=['date_time'], inplace=True)
sp500_cluster[sp500_cluster.sp500_cluster == 0].head(30)
#sp500_cluster.apply(lambda x: x.pct_change(), axis=0)
#clus_daily_rets = (sp500_cluster.sp500_index * sp500_cluster.sp500_cluster).pct_change().dropna().iloc[1:]
sp500_cluster = sp500_cluster.replace(1, 7)
sp500_cluster = sp500_cluster.replace(2, 7)
sp500_cluster = sp500_cluster.replace(3, 7)
sp500_cluster = sp500_cluster.replace(0 ,1)
sp500_cluster = sp500_cluster.replace(7, 0)
sp500_cluster.sp500_cluster = sp500_cluster.sp500_cluster.shift().fillna(0)
sp500_cluster

sp500_cluster.sp500_index = sp500_cluster.sp500_index.pct_change().fillna(0)

cluster_daily_rets = sp500_cluster.sp500_index * sp500_cluster.sp500_cluster
(1 + cluster_daily_rets).cumprod()

Metric(portfolio=(1 + cluster_daily_rets).cumprod(), freq='day').numeric_metric()

{'returns': '176.63',
 'CAGR': '4.86',
 'volatility': '0.09',
 'skewness': '-0.56',
 'kurtosis': '14.21',
 'MDD': '-0.17',
 'MDD_duration': '1230.00',
 'sharp': '0.09',
 'sortino': '0.30',
 'calmar': '0.28',
 'VaR': '-0.02',
 'VaR_ratio': '0.01',
 'CVaR': '-0.03',
 'CVaR_ratio': '0.01',
 'hit': '0.55',
 'GtP': '0.96'}

In [10]:
_list = ['returns', 'CAGR', 'volatility', 'MDD','sharp']
spcluster_bt = Metric(portfolio=(1 + cluster_daily_rets).cumprod(), freq='day').numeric_metric()

_dict = {}
for i in _list:
    _dict[i] = [spcluster_bt[i]]

spcluster_bt = pd.DataFrame(_dict, index=['spy_cluster']).T
spcluster_bt

,spy_cluster
returns,176.63
CAGR,4.86
volatility,0.09
MDD,-0.17
sharp,0.09


In [11]:
regime_df2 = pd.concat([cluster_daily_rets, regime_df], axis=1).dropna()
regime_df2.columns = ['spy_cluster', 'spy', 'deflation', 'inflation', 'recovery', 'expansion']
regime_df2.drop(columns=['spy'], inplace=True)

In [12]:
_list = ['returns', 'CAGR', 'volatility', 'MDD', 'sharp']
_regime = ['deflation',	'inflation','recovery',	'expansion']

_dict2 = []
for i in _regime:
    temp2 = {}
    for j in _list:
        temp2[j] = [Metric(portfolio=(1 + regime_df2[regime_df2[i]==1].spy_cluster).cumprod(), freq='day').numeric_metric()[j]]
    _dict2.append(pd.DataFrame(temp2, index=[f'{i}']).T)
    
temp = pd.concat(_dict2, axis=1)
final = pd.concat([spcluster_bt, temp], axis=1)
final

/Users/jtchoi/Library/CloudStorage/GoogleDrive-jungtaek0227@gmail.com/My Drive/quant/Quant-Project/quant/backtest/metric.py:156: RuntimeWarning: divide by zero encountered in double_scalars
  return (self.CAGR(returns) - yearly_rfr) / downside_std(returns)
/Users/jtchoi/Library/CloudStorage/GoogleDrive-jungtaek0227@gmail.com/My Drive/quant/Quant-Project/quant/backtest/metric.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  calmar = - self.CAGR(returns) / dd.min()
/Users/jtchoi/Library/CloudStorage/GoogleDrive-jungtaek0227@gmail.com/My Drive/quant/Quant-Project/quant/backtest/metric.py:225: RuntimeWarning: divide by zero encountered in double_scalars
  ratio = -returns.mean() / self.VaR(returns, delta=delta)
/Users/jtchoi/Library/CloudStorage/GoogleDrive-jungtaek0227@gmail.com/My Drive/quant/Quant-Project/quant/backtest/metric.py:263: RuntimeWarning: divide by zero encountered in double_scalars
  ratio = -returns.mean() / self.CVaR(returns, delta=delta)
/Users/jtch

,spy_cluster,deflation,inflation,recovery,expansion
returns,176.63,95.28,103.15,99.49,97.86
CAGR,4.86,-42.53,605.63,-4.66,-21.09
volatility,0.09,0.09,0.25,0.04,0.07
MDD,-0.17,-0.05,0.00,-0.01,-0.02
sharp,0.09,-4.98,24.06,-2.19,-3.35


In [13]:
sp500#.loc[sp500_cluster[sp500_cluster.sp500_cluster == 0].index[4:]]

2011-01-31    102.406494
2011-02-28    105.963814
2011-03-31    105.976562
2011-04-30    109.045815
2011-05-31    107.822876
                 ...    
2022-08-31    391.764313
2022-09-30    355.547302
2022-10-31    384.444611
2022-11-30    405.816467
2022-12-31    382.429993
Freq: M, Name: Adj Close, Length: 144, dtype: float64

In [14]:
(1 + clus_daily_rets).cumprod().plot()

NameError: name 'clus_daily_rets' is not defined